In [23]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent


# ====== 工具1：按行业搜索公司 ======
@tool
def search_companies_by_industry(industry: str) -> str:
    """输入行业名称，返回一个公司列表ID。必须先用此工具获取 ID，才能用 search_people_by_company_list 查询人员。"""
    return "company_list_123"


# ====== 工具2：根据公司列表ID和多个职位关键字搜索人 ======
@tool
def search_people_by_company_list(company_list_id: str, job_titles: list) -> list:
    """
    输入公司列表ID和多个职位关键字，批量查找这些公司的人。
    job_titles 示例: ["软件工程师", "后端开发", "研发工程师"]
    """
    results = []
    for title in job_titles:
        results.append({
            "name": f"Test-{title}",
            "title": title,
            "company": "DemoCorp"
        })
    return results


# ====== 工具3：用 LLM 扩展职位名称 ======
@tool
def expand_job_title(base_title: str) -> list:
    """
    使用大模型，根据一个职位名称生成相关职位名称列表。
    例如: "软件工程师" → ["软件工程师", "后端开发", "全栈工程师", "研发工程师"]
    """
    llm = init_chat_model("gpt-4o-mini", temperature=0)
    prompt = f"""
    你是一个人力资源专家。给定职位名称 "{base_title}"，
    请生成一个包含 3~5 个相关职位名称的 JSON 数组。
    只输出 JSON 数组，不要多余文字，不要用 markdown。
    """
    resp = llm.invoke(prompt)
    try:
        print('expand_job_title:', resp.content)
        import json
        job_list = json.loads(resp.content)
        if isinstance(job_list, list):
            return job_list
    except Exception:
        raise
    return [base_title]  # fallback


# ====== 初始化模型 ======
model = init_chat_model("gpt-4o-mini", temperature=0)

# ====== 内存保存器 ======
memory = MemorySaver()

# ====== 创建 ReAct Agent ======
agent = create_react_agent(
    model=model,
    tools=[
        search_companies_by_industry,
        expand_job_title,
        search_people_by_company_list
    ],
    checkpointer=memory
)


# ====== 执行 Agent 流程并打印事件 ======
user_query = "帮我查医疗行业公司的软件工程师"
config = {"configurable": {"thread_id": "demo-thread-3"}}
events = agent.stream({"messages": [("user", user_query)]}, config)

for event in events:
    if "agent" in event:
        for msg in event["agent"]["messages"]:
            msg.pretty_print()  # Agent 中间推理

    if "tools" in event:
        for msg in event["tools"]["messages"]:
            msg.pretty_print()  # 工具输入输出


================================== Ai Message ==================================
Tool Calls:
  search_companies_by_industry (call_wupYTg7yjVRNJa4D2ky5s9rD)
 Call ID: call_wupYTg7yjVRNJa4D2ky5s9rD
  Args:
    industry: 医疗
================================= Tool Message =================================
Name: search_companies_by_industry

company_list_123
================================== Ai Message ==================================
Tool Calls:
  expand_job_title (call_uGm35FANvneEyzFiJef1h3hj)
 Call ID: call_uGm35FANvneEyzFiJef1h3hj
  Args:
    base_title: 软件工程师
expand_job_title: [
    "前端开发工程师",
    "后端开发工程师",
    "全栈工程师",
    "移动应用开发工程师",
    "软件测试工程师"
]
================================= Tool Message =================================
Name: expand_job_title

["前端开发工程师", "后端开发工程师", "全栈工程师", "移动应用开发工程师", "软件测试工程师"]
================================== Ai Message ==================================
Tool Calls:
  search_people_by_company_list (call_KKUfw7DhqrgVyfgtCTcVxT1i)
 Call ID: call_KK